In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.webdriver.common.keys import Keys
import numpy as np
import time
from datetime import datetime

In [ ]:
#List of product urls we want to track prices for
urls = ['https://www.amazon.com/Bose-QuietComfort-Bluetooth-Headphones-Noise-Cancelling/dp/B07NXDPLJ9/ref=lp_509318_1_9?s=aht&ie=UTF8&qid=1587059636&sr=1-9','https://www.amazon.com/KitchenAid-KSM150PSER-Artisan-Tilt-Head-Pouring/dp/B00005UP2P/ref=sr_1_4?dchild=1&keywords=KitchenAid+Professional+5qt+Mixer+-+KV25G0X&qid=1587059540&sr=8-4','https://www.amazon.com/NuMe-Magic-Curling-Wand/dp/B00AHH9M3W/ref=sr_1_1?dchild=1&keywords=NuMe+Magic+Curling+Wand&qid=1587059253&s=electronics&sr=8-1','https://www.amazon.com/GoPro-HERO7-Silver-Commerce-Stabilization/dp/B083C5Q8VJ/ref=sr_1_1_sspa?dchild=1&keywords=go-pro&qid=1587059076&s=electronics&sr=1-1-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUExR1U1TzNZMU5QSEJLJmVuY3J5cHRlZElkPUEwOTAwNDY0MUVMWTVVTjRSM1hHRiZlbmNyeXB0ZWRBZElkPUEwNzUyODEwMURHRUxFQjRERENIUSZ3aWRnZXROYW1lPXNwX2F0ZiZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU=','https://www.amazon.com/Nikon-FX-Format-Mirrorless-Camera-Body/dp/B07GPRSYG8/ref=sxin_6?ascsubtag=amzn1.osa.9ebb2861-3fca-4182-8a9c-cd0a935f34b7.ATVPDKIKX0DER.en_US&creativeASIN=B07GPRSYG8&cv_ct_cx=camera&cv_ct_id=amzn1.osa.9ebb2861-3fca-4182-8a9c-cd0a935f34b7.ATVPDKIKX0DER.en_US&cv_ct_pg=search&cv_ct_wn=osp-single-source&dchild=1&keywords=camera&linkCode=oas&pd_rd_i=B07GPRSYG8&pd_rd_r=9534fb16-1edc-4d78-97f4-fefbad524f21&pd_rd_w=35Wwb&pd_rd_wg=bXYWc&pf_rd_p=b9bea1c2-6981-439e-9086-1a31b4eafe7d&pf_rd_r=C0AAYYHA64G3JH834Y9H&qid=1587059009&sr=1-1-72d6bf18-a4db-4490-a794-9cd9552ac58d&tag=techradar07-20','https://www.amazon.com/Fjallraven-Classic-Backpack-Everyday-Graphite/dp/B003NF25K2/ref=sr_1_21?dchild=1&keywords=backpack&qid=1587058843&s=apparel&sr=1-21','https://www.amazon.com/Philips-328E9QJAB-Frameless-FreeSync-DisplayPort/dp/B07KQXJLF7/ref=sxin_3_ac_d_pm?ac_md=5-2-QWJvdmUgJDE1MA%3D%3D-ac_d_pm&cv_ct_cx=computer+monitor&dchild=1&keywords=computer+monitor&pd_rd_i=B07KQXJLF7&pd_rd_r=d344a2d7-a4d6-49f2-be6f-458ce3284109&pd_rd_w=R0Ywb&pd_rd_wg=46Pcg&pf_rd_p=d268c452-dec7-459f-93b2-f5478382989a&pf_rd_r=0Q9VBB1Z9SY6HYGY8TQ1&psc=1&qid=1587058964&s=home-garden&sr=1-3-22d05c05-1231-4126-b7c4-3e7a9c0027d0','https://www.amazon.com/Chefman-Liter-Electric-Glass-Kettle/dp/B07KYYF9HD/ref=sr_1_1?dchild=1&keywords=816458023023&qid=1586630945&sr=8-1', 'https://www.amazon.com/Instant-Vortex-One-Touch-Programs-Reheat/dp/B07VM1FT8W/ref=sr_1_1?dchild=1&keywords=857561008637&qid=1586627447&s=home-garden&sr=1-1', 'https://www.amazon.com/CeraVe-Foaming-Facial-Cleanser-Washing/dp/B01N1LL62W/ref=sr_1_5?dchild=1&keywords=cerave&qid=1586628117&s=home-garden&sr=8-5']

In [ ]:
'''This is persistent dictionaries: write each dictionary as a file to local computer
* Might not need this at all if final project uses jupyter notebook'''

'''
# ASIN_dict
# key: ASIN, value: UPC
try:
	with open('asin_dict.pickle', 'rb') as file:
		ASIN_dict = pickle.load(file)
except FileNotFoundError:
	ASIN_dict = dict()

# UPC_dict
# key: UPC, value: product title
try:
	with open('upc_dict.picle', 'rb') as file:
		UPC_dict = pickle.load(file)
except FileNotFoundError:
	UPC_dict = dict()
'''

In [21]:
# key: asin 
# value: upc

ASIN_dict = {}

In [4]:
amazon_name = []
amazon_price = []
scrape_time = []
target_name = []
target_price = [] 
walmart_name = []
walmart_price = []

In [5]:
def scraper(urls, path):
    
    for url in urls:
        
        driver = Chrome(executable_path= path)
        driver.get(url)
        
       
        #grab product name from amazon
        amazon_name.append(driver.find_element_by_xpath('//*[@id="productTitle"]').text)
        
        #get amazon price, if price is no longer available append 'no price'
        try:
            amazon_price.append(driver.find_element_by_xpath('//*[@id="priceblock_ourprice"]').text)
        except:
            amazon_price.append('no price')
            
        #save time of the scrape
        scrape_time.append(str(datetime.today()))
        
        #get the asin
        asin = url.split('/')[5]
        driver.quit()
        
        #if upc for a specific asin has already been looked up do not look up
        if asin in ASIN_dict:
            upc_id = ASIN_dict[asin]
            
        # Getting UPC from ASIN
        else:
            upc_driver = Chrome(executable_path=path)
            upc_driver.get('https://www.synccentric.com/features/upc-asin/')
            box = upc_driver.find_element_by_id('scrollto')
            form = box.find_element_by_class_name('form-group')
            input = form.find_element_by_name('identifier')
            input.send_keys(asin)
            input.submit()
            #wait for the webpage to load
            time.sleep(10)
            id_list = upc_driver.find_element_by_class_name('col-sm-8').find_elements_by_tag_name('strong')
            upc_id = id_list[1].get_attribute('innerHTML')
            upc_driver.quit()
            
            #add the new asin --> upc to the dict
            ASIN_dict[asin] = upc_id
            
            #sleep so you are not locked out
            if len(urls) >10:
                time.sleep(360)

        #find product on target
        target_driver = Chrome(executable_path=path)
        target_url = 'https://www.target.com/s?searchTerm=' + upc_id
        target_driver.get(target_url)
        #wait for the webpage to load
        time.sleep(10)
        
        
        
        #get target name if the item is found on target 
        try:
            name = target_driver.find_element_by_xpath('//*[@id="mainContainer"]/div[3]/div[2]/div/div[1]/div[3]/div/ul/li/div/div[2]/div/div/div/div[1]/div[1]/a').text
            target_name.append(name)
        except:
            target_name.append("no name found")
        
        #Get target price if item was found on target
        try:
            price = target_driver.find_element_by_xpath('//*[@id="mainContainer"]/div[3]/div[2]/div/div[1]/div[3]/div/ul/li/div/div[2]/div/div/div/div[2]/span').text
            target_price.append(price)
        except:
            target_price.append("no price found")
 
        target_driver.quit()
    
        #find product on walmart
        walmart_driver = Chrome(executable_path=path)
        walmart_url = 'https://www.walmart.com/search/?query=' + upc_id
        walmart_driver.get(walmart_url)
        
        names_prices = walmart_driver.find_elements_by_xpath("//div[contains(@class, 'tile-content Grid-col u-size-8-10-l list-description-wrapper')]")
        
        #get walmart name and price if product was found on walmart
        if len(names_prices) == 0:
            
            walmart_name.append('no name found')
            walmart_price.append('no price found')
        else:
            for i in names_prices:
                if 'Pack' not in i.text:
                    #if name of the product from walmart has already been looked up do not scrape again
                    
                    walmart_name.append(i.text.split('\n')[1])
                    walmart_price.append(i.text.split('\n')[9])
              
        walmart_driver.quit()

In [6]:
# while True:
#     scraper(urls)
#     time.sleep(10)

In [7]:
i = 1
while i <3:
    print(i)
    scraper(urls)
    time.sleep(10)
    i += 1

1
2


In [8]:
prices = pd.DataFrame({'amazon_name' : amazon_name,
          'amazon_price' : amazon_price,
          'scrape_time' : scrape_time,
            'target_name': target_name,
            'target_price': target_price,
             'walmart_name': walmart_name,
             'walmart_price': walmart_price}, 
         columns=['amazon_name','amazon_price', 'scrape_time','target_name','target_price','walmart_name', 'walmart_price' ])    

In [9]:
prices

,amazon_name,amazon_price,amazon_time,target_name,target_price,walmart_name,walmart_price
0,Chefman 1.7 Liter Electric Glass Tea Kettle wi...,$19.99,2020-04-11 21:02:10.310191,Chefman 1.7L Glass Electric Kettle - Clear,$19.99,Chefman 1.7 Liter Electric Glass Tea Kettle wi...,$24.99
1,"Instant Vortex 4-in-1 Air Fryer, 6 Quart, 4 On...",$99.00,2020-04-11 21:03:20.662978,Instant Pot 6qt Vortex Air Fryer,$99.99,no name found,no price found
2,CeraVe Foaming Facial Cleanser | 16 Fl Oz | Da...,$14.13,2020-04-11 21:04:34.692557,CeraVe Foaming Facial Cleanser for Normal to O...,$14.99,Cerave Oil Control Value Size Foaming Facial C...,$15.64
3,Chefman 1.7 Liter Electric Glass Tea Kettle wi...,$19.99,2020-04-11 21:05:55.926576,Chefman 1.7L Glass Electric Kettle - Clear,$19.99,Chefman 1.7 Liter Electric Glass Tea Kettle wi...,$24.99
4,"Instant Vortex 4-in-1 Air Fryer, 6 Quart, 4 On...",$99.00,2020-04-11 21:07:20.548234,Instant Pot 6qt Vortex Air Fryer,$99.99,no name found,no price found
5,CeraVe Foaming Facial Cleanser | 16 Fl Oz | Da...,$14.13,2020-04-11 21:08:33.725646,CeraVe Foaming Facial Cleanser for Normal to O...,$14.99,Cerave Oil Control Value Size Foaming Facial C...,$15.64


In [3]:
# MAIN

buyer_type = input('Are you an Informed Buyer (I) or a Window Shopper (W)? ').strip().upper()

# This whole if can be converted to a function to add more URLs to the urls list

if buyer_type == 'I':
	basket = []
	item = input('Paste Amazon link of the product you want to track or press Enter key when done ')
	while True:
		if item == '':
			break
		else:		
			basket.append(item)
			item = input('Paste Amazon link of the product you want to track or press Enter key when done ')
	# print('Your basket: ', basket)	

# 	informed_buyer(basket)


else:    # window shopper
	pass

Are you an Informed Buyer (I) or a Window Shopper (W)? I
Paste Amazon link of the product you want to track or press Enter key when done 
